In [19]:
import sklearn
import sys
print(f'sklearn version : {sklearn.__version__}')
print(f'python version : {sys.version}')

sklearn version : 0.23.2
python version : 3.7.8 | packaged by conda-forge | (default, Jul 31 2020, 01:53:57) [MSC v.1916 64 bit (AMD64)]


In [20]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=RuntimeWarning)
from IPython.display import HTML, display # jupyter 사진넣기
import time                      # excution time 계산
from datetime import datetime   # system time 
import json                     # json save
# from imblearn.over_sampling import SMOTE
import sklearn
from sklearn import tree                  # 결정트리 모형  # Classifier tree
from sklearn.inspection import permutation_importance  # feature 중요도
from sklearn.pipeline import Pipeline
from scipy import stats
from scipy.stats import randint, uniform, loguniform
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Binarizer, PolynomialFeatures
from sklearn.metrics import (accuracy_score,precision_score,recall_score,f1_score,roc_auc_score,confusion_matrix,
                             precision_recall_curve,roc_curve,mean_squared_error,mean_absolute_error,r2_score)
from sklearn.model_selection import learning_curve, ShuffleSplit

from sklearn.linear_model import LogisticRegression,SGDClassifier,LinearRegression,SGDRegressor,Ridge,Lasso,ElasticNet
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, SVR, LinearSVC,LinearSVR
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, ExtraTreeRegressor, plot_tree 
from sklearn.ensemble import (VotingClassifier, BaggingClassifier, RandomForestClassifier, AdaBoostClassifier,
                            ExtraTreesClassifier,GradientBoostingClassifier,RandomForestRegressor,GradientBoostingRegressor)
from xgboost import XGBClassifier, XGBRegressor     # 사이킷런 wrapper
from lightgbm import LGBMClassifier, LGBMRegressor   # 사이킷런 wrapper

from catboost import CatBoostClassifier, CatBoostRegressor
from ngboost import NGBClassifier, NGBRegressor


import pickle                         # 모델 저장
import matplotlib as mpl              # 한글깨짐
import matplotlib.font_manager as fm  # 한글깨짐
import matplotlib.font_manager        # 한글깨짐


## Plot

In [21]:
# linux 
font_dirs = ['/home/sch/anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/fonts/ttf', ]
font_files = fm.findSystemFonts(fontpaths=font_dirs)
font_list = fm.createFontList(font_files)
fm.fontManager.ttflist.extend(font_list)
[f.name for f in matplotlib.font_manager.fontManager.ttflist if 'Nanum' in f.name][:3]

[]

In [22]:
# 모델에 대한 성능평가지표개선을 위한 threshold 선택관련 plot
def precision_recall_curve_plot(y_test, pos_proba, model_name=None,time_name=None ): # pred_proba positive값
    # threshold ndarray와 이 threshold에 따른 정밀도, 재현율 ndarray 추출.
    precisions, recalls, thresholds = precision_recall_curve(y_test, pos_proba)

    # X축을 threshold값으로, Y축은 정밀도, 재현율 값으로 각각 Plot 수행. 정밀도는 점선으로 표시
    fig = plt.figure(figsize=(8, 6))
    # 한글깨짐 윈도우(window)
    font_name = mpl.font_manager.FontProperties(fname='C:/Windows/Fonts/malgun.ttf').get_name() # 폴더구분자 python / 임
    mpl.rc('font', family=font_name) # ahronbd.ttf
    
#     mpl.rcParams['font.family'] = 'NanumGothic'
#     mpl.rc('axes', unicode_minus=False)
    threshold_boundary = thresholds.shape[0]
    plt.plot(thresholds, precisions[0:threshold_boundary],linestyle='--', label='precision')
    plt.plot(thresholds, recalls[0:threshold_boundary], label='recall')

    # threshold 값 X 축의 Scale을 0.1 단위로 변경
    start, end = plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1), 2))

    # x축, y축 label과 legend, 그리고 grid 설정
    plt.title(f"{data_name} {model_name} Precision and Recall Trade off Curve ")
    plt.xlabel('Threshold value)')
    plt.ylabel('Precision and Recall value')
    plt.legend()
    plt.grid()
    plt.savefig(f'./img/{data_name}_{model_name}_{time_name}_PR.png', bbox_inches='tight')
#         plt.close(fig)
    plt.show()

def roc_curve_plot(y_test, pos_proba, model_name=None,time_name=None): # pred_proba positive값
    # 한글깨짐 윈도우(window)
    font_name = mpl.font_manager.FontProperties(fname='C:/Windows/Fonts/malgun.ttf').get_name() # 폴더구분자 python / 임
    mpl.rc('font', family=font_name) # ahronbd.ttf
    
#     mpl.rcParams['font.family'] = 'NanumGothic'
#     mpl.rc('axes', unicode_minus=False)
    # 임곗값에 따른 FPR, TPR 값을 반환 받음.
    fprs, tprs, thresholds = roc_curve(y_test, pos_proba)

    fig = plt.figure(figsize=(8, 6))
    # ROC Curve를 plot 곡선으로 그림.
    plt.plot(fprs, tprs, label='ROC')
    # 가운데 대각선 직선을 그림.
    plt.plot([0, 1], [0, 1], 'k--', label='Random')

    # FPR X 축의 Scale을 0.1 단위로 변경, X,Y 축명 설정등
    start, end = plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1), 2))
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.title(f'{data_name} {model_name} Model ROC Curve')
    plt.xlabel('FPR( 1 - Sensitivity )')
    plt.ylabel('TPR( Recall )')
    plt.legend()
    plt.savefig(f'./img/{data_name}_{model_name}_{time_name}_ROC.png', bbox_inches='tight')
#         plt.close(fig)
    plt.show()

# 선형기반 회귀계수 coef_ (lr, Ridge, Lasso, ) # gridsearchCV 에서는 없음
def Plot_coef_(model, indexs, model_name, time_name):
    # 한글깨짐 윈도우(window)
    font_name = mpl.font_manager.FontProperties(fname='C:/Windows/Fonts/malgun.ttf').get_name() # 폴더구분자 python / 임
    mpl.rc('font', family=font_name) # ahronbd.ttf
    # 리눅스
#     mpl.rcParams['font.family'] = 'NanumGothic'
#     mpl.rc('axes', unicode_minus=False)
    df = pd.Series(model.coef_[0], index=indexs) 
    df = df.sort_values(ascending=False) # [:20]
    plt.figure(figsize=(8,6))
    sns.barplot(x=df, y=df.index)
    plt.title(f'{data_name} {model_name} coef_ ')
#     plt.xlabel(f'{model_params}')
    plt.savefig(f"./imgSel/{data_name}_{model_name}_{time_name}_coef_.png", bbox_inches='tight') # dpi=100
#     plt.close(fig)
    plt.show()  
    
# 회귀트리(dt, rf, xgb, lgbm ) 이용시 feature_importances_  # gridsearchCV 에서는 없음
def Plot_feature_importance(model, indexs, model_name, time_name):
    # 한글깨짐 윈도우(window)
    font_name = mpl.font_manager.FontProperties(fname='C:/Windows/Fonts/malgun.ttf').get_name() # 폴더구분자 python / 임
    mpl.rc('font', family=font_name) # ahronbd.ttf
    # 리눅스
#     mpl.rcParams['font.family'] = 'NanumGothic'
#     mpl.rc('axes', unicode_minus=False)
    df = pd.Series(model.feature_importances_, index=indexs) 
    df = df.sort_values(ascending=False) # [:20]
    plt.figure(figsize=(8,6))
    sns.barplot(x=df, y=df.index)
    plt.title(f'{data_name} {model_name} feature_importance')
#     plt.xlabel(f'{model_params}')
    plt.savefig(f"./imgSel/{data_name}_{model_name}_{time_name}_FI.png", bbox_inches='tight') # dpi=100
#     plt.close(fig)
    plt.show()  
    
    # permutation_importance : sklearn 0.23.2이상
def Plot_permutation_importance(importance, indexs, model_name, time_name):
    # 한글깨짐 윈도우(window)
    font_name = mpl.font_manager.FontProperties(fname='C:/Windows/Fonts/malgun.ttf').get_name() # 폴더구분자 python / 임
    mpl.rc('font', family=font_name) # ahronbd.ttf
#     mpl.rcParams['font.family'] = 'NanumGothic'
#     mpl.rc('axes', unicode_minus=False)

    df = pd.Series(importance, index=indexs)
    df_20 = df.sort_values(ascending=False)#[:20]
    plt.figure(figsize=(8,6))
    sns.barplot(x=df_20, y=df_20.index)
    plt.title(f'{data_name} {model_name} permutation_importance')
    plt.savefig(f"./imgSel/{data_name}_{model_name}_{time_name}_PI.png", bbox_inches='tight') # dpi=100
#     plt.close(fig)
    plt.show() 
        
 # 결정트리 모형도 sklearn 0.23.1   (lr, rf 사용못함, CART 가능) : 수행시간이 많이 걸림
def Plot_tree_(model,model_name,time_name):
    # 한글깨짐 윈도우(window)
    font_name = mpl.font_manager.FontProperties(fname='C:/Windows/Fonts/malgun.ttf').get_name() # 폴더구분자 python / 임
    mpl.rc('font', family=font_name) # ahronbd.ttf
#     mpl.rcParams['font.family'] = 'NanumGothic'
#     mpl.rc('axes', unicode_minus=False)
    plt.figure(figsize=(12,8))
    tree.plot_tree(model, filled=True)
    plt.title(f'{data_name} {model_name} tree')
    plt.savefig(f"./imgTree/{data_name}_{model_name}_{time_name}_tree.png", bbox_inches='tight')
#     plt.close(fig)
    plt.show()    

In [23]:
# 학습곡선
def Plot_learning_curve(model, model_name, time_name,  X, y, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5)):
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    train_sizes, train_scores, test_scores, fit_times, _ = learning_curve(
            model, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, scoring = 'neg_mean_squared_error', return_times=True)
    train_scores_mean = np.mean(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    plt.figure(figsize=(8,6))
    plt.title(f'{data_name} {model_name} learing curve')
    plt.grid()
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",label="Training error")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",label="Cross-validation error")
    plt.legend(loc="best")
    plt.xlabel("Training ")
    plt.ylabel("neg_mean_squared_error")
    plt.savefig(f"./imgLearnCurve/{data_name}_{model_name}_{time_name}_LC.png", bbox_inches='tight')
#     plt.close(fig)
    plt.show()

In [24]:
# cv_results_데이터로 GridSearchCV 곡선
def plot_grid_curve(model, grid_params, model_name,time_name):
    df = pd.DataFrame(model.cv_results_)
    results = ['mean_test_score','mean_train_score'] 
    fig, axes = plt.subplots(1, len(grid_params), figsize = (3*len(grid_params), 4) )  # sharey='row'
    axes[0].set_ylabel("Score", fontsize=10)
    for idx, (param_name, param_range) in enumerate(grid_params.items()):
        grouped_df = df.groupby(f'param_{param_name}')[results].agg({'mean_train_score': 'mean', 'mean_test_score': 'mean'})
        previous_group = df.groupby(f'param_{param_name}')[results]
        axes[idx].set_xlabel(param_name, fontsize=10)
#         axes[idx].set_ylim(0.0, 1.1)
        lw = 2 # 선의 굵기
        axes[idx].plot(param_range, grouped_df['mean_train_score'], label="Training score", color="darkorange", lw=lw)
        axes[idx].plot(param_range, grouped_df['mean_test_score'], label="Cross-validation score",  color="navy", lw=lw)
#     handles, labels = axes[0].get_legend_handles_labels()
    fig.suptitle(f'{data_name} {model_name} Validation curves', fontsize=20)
    plt.legend(loc = "best")
    fig.savefig(f"./imgGridCurve/{data_name}_{model_name}_{time_name}_GC.png", bbox_inches='tight')
    fig.subplots_adjust(bottom=0.25, top=0.85)  
#     plt.close(fig)
    plt.tight_layout()
    plt.show()    

### 2.  metric 함수

In [25]:
## clf
# 다중분류에서는 confusion_matrix, accuracy, roc_auc 만 나온다
def get_clf_eval_poly(y_test, pred, pos_proba=None):
    metrics_dict={}
    confusion = confusion_matrix( y_test, pred)
    metrics_dict['confusion_matrix'] = confusion.tolist() # confusion nd.array  --> list
    accuracy = accuracy_score(y_test , pred)
    metrics_dict['accuracy'] = accuracy
    roc_auc = roc_auc_score(y_test, pos_proba) 
    metrics_dict['auc_score'] = auc_score
    print(f'오차 행렬 :\n {confusion}')
    print(f" 정확도: {accuracy:.4f}, auc_score:{roc_auc:.4f}")
    return metrics_dict

# 다중분류에서는 confusion_matrix, accuracy만 나온다
# 이진분류에서 성능평가지표 종합 함수
def get_clf_eval(y_test=None, pred=None, pos_proba=None): # 권철민 저 책자에 누락 pred_proba
    metrics_dict={}
    confusion = confusion_matrix(y_test, pred)
    metrics_dict['confusion_matrix'] = confusion.tolist()
    accuracy = accuracy_score(y_test, pred)
    metrics_dict['accuracy'] = accuracy
    precision = precision_score(y_test, pred)  # average binary # multi [None, 'micro', 'macro', 'weighted'].
    metrics_dict['precision'] = precision
    recall = recall_score(y_test, pred )     
    metrics_dict['recall'] = recall
    f1 = f1_score(y_test, pred )
    metrics_dict['f1'] = f1
    roc_auc = roc_auc_score(y_test, pos_proba) # 권철민 저 책자에 오기 pred --> pred_proba  # average='macro'
    metrics_dict['roc_auc'] = roc_auc
    # print(f'오차 행렬 :\n {confusion}')
    # print(f" 정확도: {accuracy:.4f}, 정밀도: {precision:.4f}, 재현율: {recall:.4f}, auc_score:{roc_auc:.4f}")
    print(f" 정확도: {accuracy:.4f}, 정밀도: {precision:.4f}, 재현율: {recall:.4f}, f1_score: {f1:.4f}, auc_score:{roc_auc:.4f}")
    return metrics_dict

## reg 
# log 값 변환 시 NaN등의 이슈로 log() 가 아닌 log1p() 를 이용하여 RMSLE 계산
def rmsle(y_test, pred):
    log_y = np.log1p(y_test)
    log_pred = np.log1p(pred)
    squared_error = (log_y - log_pred) ** 2
    rmsle = np.sqrt(np.mean(squared_error))
    return rmsle

# 사이킷런의 mean_square_error() 를 이용하여 np.sqrt로 RMSE 계산
def rmse(y_test,pred):
    return np.sqrt(mean_squared_error(y_test,pred))

# MAE, MSE, RMSE, RMSLE 를 모두 계산 
def get_reg_eval(y_test, pred):
    metrics_dict={}
    mae_val = mean_absolute_error(y_test,pred)
    metrics_dict['MAE'] = mae_val
    mse_val = mean_squared_error(y_test,pred)
    metrics_dict['MSE'] = mse_val
    rmse_val = rmse(y_test,pred)
    metrics_dict['RMSE'] = rmse_val
    rmsle_val = rmsle(y_test,pred)
    metrics_dict['RMSLE'] = rmsle_val    
    r2 = r2_score(y_test,pred)
    metrics_dict['R2'] = r2   
    # adjusted_r2 = 1-(1-r2_score(y_test, pred))*((len(X_test)-1)/(len(X_test)-len(X_test[0])-1)))
    print(f'MAE: {mae_val:.3F}, MSE: {mse_val:.3F}, RMSE: {rmse_val:.3F}, RMSLE: {rmsle_val:.3f}, R2: {r2:.3f}\n')
    return metrics_dict

### 3. model train, metric 호출 

### 3-3 metricPlot  ## 2 metric함수 호출

In [26]:
def metricPlot(model, model_name, time_name, start_model_time, X_train,y_train,X_test, y_test):
    linear_model=['LogisticRegression','SGDClassifier','LinearSVC','LinearRegression','LinearSVR','SGDRegressor','Ridge','Lasso','ElasticNet']
    pred = model.predict(X_test)
    if(est =="clf"):
        if model_name in ['LinearSVC']:
            pred_proba = model._predict_proba_lr(X_test) 
        else:
            pred_proba = model.predict_proba(X_test) 
    if yLog : #is_expm1 <= targetlog변환시 복원
        y_test = np.expm1(y_test)
        pred = np.expm1(pred)  
    if (est == 'clf'):
        metrics_dict = get_clf_eval(y_test, pred, pred_proba[:,1])
    elif (est == 'reg'):
        metrics_dict = get_reg_eval(y_test, pred)
    spend_metric_time=(time.time() - start_model_time)
    print(f'{model_name} metric 수행시간 : {int(spend_metric_time//60)}분 {spend_metric_time%60:.2f}초\n')
    if (plot == 1) :
        Plot_learning_curve(model,model_name,time_name,X_train,y_train,n_jobs=-1,train_sizes=np.linspace(.1,1.0,5))
        if (est =="clf"):
            precision_recall_curve_plot(y_test, pred_proba[:,1], model_name, time_name )
            roc_curve_plot(y_test, pred_proba[:,1], model_name, time_name)
        if model_name in linear_model:
            Plot_coef_(model, X_train.columns, model_name, time_name) # 회귀계수 
        else : 
# [KNeighborsClassifier,DecisionTreeClassifier,RandomForestClassifier,GradientBoostingClassifier,XGBClassifier,LGBMClassifier]:
            Plot_feature_importance(model, X_train.columns, model_name, time_name) # 회귀트리 # lr x
        # permutation_importance
        if (est == 'clf'):
            results = permutation_importance(model, X_test, y_test, scoring='f1')
        elif (est == 'reg'):
            results = permutation_importance(model, X_test, y_test, scoring='r2')
        importance = results.importances_mean
        Plot_permutation_importance(importance, X_test.columns, model_name, time_name)
    return metrics_dict

In [27]:
# get_params()관련 LGBM기준 (XGB유사) # sklearn version별 api 다름
# max_depth log2(데이터수) # bostonHouse 506=>8, # bike 10886=>13 # AmesHouse 1460=>10 # project 797만=>22
# num_leaves  : 2^(max_depth) 작아야 함  최대 리프노드 갯수 = max_leaf_nodes(dt, rf)
# min_child_samples : Leaf node가 되기 위한 최소한 데이터 개체수  = min_samples_leaf  # min_samples_split (dt,rf 분리 최소 데이터)
# min_child_weight = min_child_leaf (dt, rf) *gbm은 없음    
# subsample (gbm,xgb,lgbm) 데이터 샘플링 비율 *lgbm은 subsample_for_bin 20000, subsample_freq 0
# colsample_bytree = max_features (dt, rf, gbm) 개별 트리를 학습할 때마다 무작위로 선택하는 feature 비율을 제어 (feature 많을 경우) * 
    # XGB : colsample_bylevel, colsample_bynode, colsample_bytree
# reg_alpha : L1 규제(제거) # reg_lambda : L2 규제(최소화) 
# 영혼까지 learning_rate 내리고,  n_estimators(rf,gbm,xgb,lgbm) 올리고
# stats.uniform(loc, scale) # subsample[0,1]  loc + scale =1 범위를 넘어가면 안됨
def get_params(model_name):  # param_grid 2개항목 이상일때만 plot_grid_curve 가능   # l1_ratio  0 ~ 1 사이값
    if (grid == 2) : # RandomSearch    속도느링 LogisticRegression  GradientBoostingClassifier
        # clf
        if model_name == 'LogisticRegression': # 속도느림 #"solver":['lbfgs','auto']
#             param_grid = { 'C': stats.loguniform(1e-1, 1e2),'l1_ratio': stats.uniform(0, 1), "max_iter":stats.randint(100,500) }
            param_grid = { 'C': stats.loguniform(1e-1, 1e2), "max_iter":stats.randint(100,500) }
        elif model_name == 'LinearSVC': # 'loss' ['hinge',''] # C 커지면 곡선 -> 직선 # tol(허용오차)
            param_grid = { 'C': stats.loguniform(1e-1, 1e2),"max_iter":stats.randint(100,500) }
        elif model_name == 'KNeighborsClassifier':#'algorithm':'auto' # p:1 manhattan_distance, 2 euclidean_distance 
            param_grid ={'n_neighbors':[5,10],'p':[1,2]}
        elif model_name == 'SGDClassifier':#early_stopping:False,'learning_rate':'optimal','penalty':'l2','loss':'hinge','eta0':[0,1],
            param_grid = { 'alpha':stats.loguniform(1e-4,1e2),'l1_ratio':stats.uniform(0,1),'average':[True,False],
                            'epsilon':stats.loguniform(1e-1, 1e1),'max_iter':stats.randint(100,500)}
        elif model_name == 'DecisionTreeClassifier': 
            param_grid = {"max_depth": stats.randint(5, 50), "max_features": stats.randint(2, 24),
                "min_samples_split":stats.randint(10,100),"min_samples_leaf":stats.randint(10,100),"criterion":["gini","entropy"]}
        elif model_name == 'RandomForestClassifier': # n_estimators=100
            param_grid = {"max_depth":stats.randint(5, 50),"max_features":stats.randint(2, 24),"min_samples_split":stats.randint(10,100),
                          "min_samples_leaf":stats.randint(10,100),"bootstrap":[True,False],"criterion":["gini","entropy"]}
        elif model_name == 'GradientBoostingClassifier':#속도느림#n_estimators=100 learning_rate:0.1 subsample:1 ccp_alpha:0 n_features
            param_grid={'max_depth':stats.randint(5,50),'min_samples_leaf':stats.randint(10,500),'max_features':stats.randint(5,24)}
        elif model_name == 'XGBClassifier': #min_child_samples, num_leaves 없음 #n_estimators=100 learning_rate:0.1 
            param_grid={'n_estimators':stats.randint(200,1000),'learning_rate':stats.uniform(0.01,0.6),'subsample':stats.uniform(0.3,0.7),
            "max_depth":stats.randint(5,50),'colsample_bytree':stats.uniform(0.3,0.7),'min_child_weight':stats.loguniform(1e-3, 1e3)}
        elif model_name == 'LGBMClassifier':#n_estimators=100 learning_rate:0.1
            param_grid={'n_estimators': stats.randint(200, 1000),'learning_rate': stats.uniform(0.01,0.6),
                        'min_child_samples':stats.randint(10,500),'colsample_bytree':stats.uniform(loc=0.4,scale=0.6),
                        'num_leaves': stats.randint(32,1000),'subsample':stats.uniform(loc=0.2, scale=0.8),
        'min_child_weight': stats.loguniform(1e-3,1e3),'reg_alpha':stats.loguniform(1e-3,1e2),'reg_lambda':stats.loguniform(1e-3,1e3)}

        # reg
        elif model_name == 'LinearRegression': # normalize True => L2 norm 1
            param_grid  = {'copy_X': [True, False], 'fit_intercept': [True,False], 'normalize':[True,False] }
        elif model_name == 'LinearSVR': 
            param_grid = {'C':stats.loguniform(1e-1,1e2),'intercept_scaling':loguniform(1e-1,1e0),"max_iter":stats.randint(100,500)}
        elif model_name == 'SGDRegressor':
            param_grid = {'alpha':stats.loguniform(1e-4,1e2),'l1_ratio':stats.uniform(0,1),'average':[True,False],
                            'epsilon':stats.loguniform(1e-1, 1e1),'max_iter':stats.randint(100,500)}
        elif model_name == 'Ridge': 
            param_grid = {'alpha': stats.loguniform(1e-4,1e2),'fit_intercept': [True,False] } 
        elif model_name == 'Lasso': 
            param_grid = {'alpha': stats.loguniform(1e-4,1e2),'fit_intercept': [True,False] } 
        elif model_name == 'ElasticNet': # if L1 a, if L2 b  => alpha = a+b, l1_ratio = a/(a+b)  # if l1_ratio=1 => a=1, b=0
            param_grid = {'alpha': stats.loguniform(1e-4,1e2),'l1_ratio': stats.uniform(0,1)} 
        elif model_name == 'DecisionTreeRegressor':
            param_grid = {"max_depth": stats.randint(5, 50), "max_features": stats.randint(2, 24),
              "min_samples_split": stats.randint(10, 100), "min_samples_leaf": stats.randint(10, 100)}
        elif model_name == 'RandomForestRegressor': # cpu 병렬지원(n_jobs=-1, 100% 가동)
            param_grid = {"max_depth":  stats.randint(5, 50),"max_features":stats.randint(2, 24),"min_samples_split":stats.randint(10,100),
                          "min_samples_leaf":stats.randint(10,100)}
        elif model_name == 'GradientBoostingRegressor': # param많으면 학습시간이 많이 걸림(순차 시행, cpu 1개만 수행)
            param_grid={ 'max_depth' : stats.randint(5, 50),'min_samples_leaf': stats.randint(10,500),'max_features':  stats.randint(5,24)}
        elif model_name == 'XGBRegressor': #min_child_samples없음 # num_leaves 없음 # cpu 병렬지원(n_jobs=-1, auto) # 시간소요
            param_grid={'n_estimators': stats.randint(200,1000),'learning_rate': stats.uniform(0.01,0.6),'subsample':stats.uniform(0.2,0.8),
                "max_depth":stats.randint(5,50),'colsample_bytree':stats.uniform(0.2,0.8),'min_child_weight':stats.loguniform(1e-3, 1e3) }
        elif model_name == 'LGBMRegressor':  # cpu 병렬지원(n_jobs=-1, 100% 가동) # 매우빠름
            param_grid={'n_estimators':stats.randint(200,1000),'learning_rate':stats.uniform(0.01,0.6),'min_child_samples':stats.randint(10,500),
                'colsample_bytree':stats.uniform(loc=0.4,scale=0.6),'num_leaves': stats.randint(32,1000),'subsample':stats.uniform(0.2,0.8),
                'min_child_weight':stats.loguniform(1e-3,1e3),'reg_alpha':stats.loguniform(1e-3,1e2),'reg_lambda':stats.loguniform(1e-3,1e3)}
        else:
            param_grid = None
        return param_grid
        
    elif (grid == 3) : #GridSearch     
        # clf
        if model_name == 'LogisticRegression': # "solver":['lbfgs','auto']
            param_grid  = {'C': [0.1,0.2,0.3],"l1_ratio": [0.4,0.6,0.8],"max_iter":[400,500,600]}
        elif model_name == 'LinearSVC': # 'loss' ['hinge',''] # C 커지면 곡선 -> 직선 # tol(허용오차)
            param_grid ={'C': [0.2,0.3,0.4],"max_iter":[200,300,400] }
        elif model_name == 'KNeighborsClassifier':#'algorithm':'auto' # p:1 manhattan_distance, 2 euclidean_distance 
            param_grid ={'n_neighbors':[5,10],'p':[1,2]}
        elif model_name == 'SGDClassifier':#early_stopping:False,'learning_rate':'optimal','penalty':'l2','loss':'hinge','eta0':[0.0,1.0]
            param_grid ={'alpha': [0.00001,0.001,0.1],"l1_ratio":[0.1,0.5,1],'epsilon':[0.1,0.5,1],'max_iter':[100,300,500]} # 빠름
        elif model_name == 'DecisionTreeClassifier': 
            param_grid={'max_depth':[25,35,45],'min_samples_leaf':[50,100,150],'max_features':[10,20,30],'min_samples_split':[50,100,150]}
        elif model_name == 'RandomForestClassifier': # n_estimators=100
            param_grid={'max_depth':[25,35,45],'min_samples_leaf':[10,20,30],'max_features':[5,10,20],'min_samples_split':[40,55,70]}
        elif model_name == 'GradientBoostingClassifier':#속도느림 #n_estimators=100learning_rate:0.1subsample:1.0 ccp_alpha:0.0 n_features
            param_grid={'max_depth':[20,30],'min_samples_leaf':[200,330, 400],'max_features':[10,15]}
        elif model_name == 'XGBClassifier': #min_child_samples, num_leaves 없음 #n_estimators=100 learning_rate:0.1 
            param_grid={# 'n_estimators':[400,500,600], 
                        'learning_rate':[0.08],
                        'max_depth':[30,40,50],'min_child_weight':[100,500,1000],
                        'colsample_bytree':[0.2,0.4,0.6],'subsample':[0.5,0.7,0.9]}
                        #'reg_alpha':[0,0.5,1],'reg_lambda':[0,0.5,1]}
        elif model_name == 'LGBMClassifier':#n_estimators=100 learning_rate:0.1
            param_grid={'n_estimators':[800],
                         'learning_rate':[0.3],
                        'min_child_samples':[300,500,700],'min_child_weight':[700,900,1000],
                        'num_leaves':[200,250,300],'colsample_bytree':[0.5,0.75,1],'subsample':[0.4,0.55,0.7],
                         'reg_alpha': [0.057],'reg_lambda':[0.021]}
      
        # reg
        elif model_name == 'LinearRegression':
            param_grid  = {'n_jobs' : [-1, 20 ], 'fit_intercept' : [True, False]}
        elif model_name == 'LinearSVR':#degree,C,kernal,epsilon,gamma,tol(허용오차)#degree차원 #C커지면곡선->직선 #gamma커지면과적합
            param_grid ={'C': [1, 10], 'max_iter' : [100, 1000]}
        elif model_name == 'SGDRegressor':#early_stopping:False,'learning_rate': 'optimal','penalty': 'l2','loss': 'hinge','eta0':[0.0,1.0],
            param_grid ={'alpha': [0.0001,0.1],"l1_ratio":[0.01,0.5],'epsilon':[0.1,1],'max_iter':[100,500]}
        elif model_name == 'Ridge': # L2규제(feature 가중치(베타)최소화)
            param_grid ={'alpha': [0.01, 0.1, 1, 10, 100, 500], 'solver':['auto','svd']}
        elif model_name == 'Lasso': # L1규제 (feature 가중치 제거)
            param_grid ={'alpha': [0.01, 0.1, 1, 10, 100, 500], 'max_iter' : [100, 1000]}
        elif model_name == 'ElasticNet': # if L1 a, if L2 b  => alpha = a+b, l1_ratio = a/(a+b)  # if l1_ratio=1 => a=1, b=0
            param_grid ={'alpha': [0.1, 1, 10, 100, 500], "l1_ratio": [0.1, 0.3, 0.5, 0.7, 0.9]}
        elif model_name == 'DecisionTreeRegressor':
            param_grid={'max_depth':[15,22,30],'min_samples_leaf':[100,200,300],'min_samples_split':[200,500,1000]}
        elif model_name == 'RandomForestRegressor': # cpu 병렬지원(n_jobs=-1, 100% 가동)
            param_grid={'max_depth':[15,22,30],'min_samples_leaf':[100,200,300],'min_samples_split':[200,500,1000]}
        elif model_name == 'GradientBoostingRegressor': # param많으면 학습시간이 많이 걸림(순차 시행, cpu 1개만 수행)
            param_grid={'max_depth':[15,22,30],'min_samples_leaf':[100,200,300],'min_samples_split':[200,500,1000]}
        elif model_name == 'XGBRegressor': #min_child_samples없음 # num_leaves 없음 # cpu 병렬지원(n_jobs=-1, auto) # 시간소요
            param_grid={'max_depth':[15,22,30],'min_child_weight':[0.1,1,5],
                        'colsample_bytree':[0.5,0.75,0.95],'subsample':[0.6,0.8,1]}
        elif model_name == 'LGBMRegressor':  # cpu 병렬지원(n_jobs=-1, 100% 가동) # 매우빠름
            param_grid={'min_child_samples':[20,60,100],'num_leaves':[32,64,128],
                        'colsample_bytree':[0.5,0.75,1],'subsample':[0.6,0.8,1]}
                
        else:
            param_grid = None
        return param_grid

### 3-2 get_model_train_eval  # 3-3 metricPlot 호출

In [28]:
# 모델과 학습/테스트 데이터 셋을 입력하면 학습 -> GridSerchCV 수행 + 성능 평가(metric) 등 반환
def get_model_train_eval(model,model_name,time_name,start_model_time,X_train,X_test,y_train,y_test,eval_X=None,eval_y=None):
# def get_model_train_eval(model,model_name,time_name,start_model_time,X_train,X_test,y_train,y_test):
    early_stopping_modellist = ['XGBClassifier','LGBMClassifier','XGBRegressor','LGBMRegressor'] # SGD 자체적
    print('###',model.__class__.__name__,'###')
    if (grid == 2) :  # RandomSearchCV
        param_grid  = get_params(model_name)
        print(f'RandomSearchCV 입력 파라미터:{param_grid}')
        if (est == 'clf'): #  분류 scoring : accuracy, f1, neg_log_loss, roc_auc
            model=RandomizedSearchCV(model,param_distributions=param_grid,scoring='roc_auc',return_train_score=False, n_iter=10)
        elif (est =='reg'):
            model=RandomizedSearchCV(model,param_distributions=param_grid,scoring='neg_mean_squared_error',return_train_score=True,n_iter=5)
        if (model_name in early_stopping_modellist):
            model.fit(X_train, y_train,early_stopping_rounds=20,eval_metric="logloss",eval_set=[(eval_X,eval_y)],verbose=False)
        else:
            model.fit(X_train, y_train) 
        best_params = model.best_params_
        print(f'RandomSearchCV 최적 파라미터:{ best_params}\n')
        # RandomSearch plot_grid_curve : error (because of param_grid scipy.stats)
        model = model.best_estimator_
        metrics_dict = metricPlot(model,model_name,time_name,start_model_time,X_train,y_train,X_test,y_test)
        return model, best_params, metrics_dict

    elif (grid == 3) : # GridSearchCV
        param_grid  = get_params(model_name)
        print(f'GridSearchCV 입력 파라미터:{param_grid}')
        if (est == 'clf'): #  분류 scoring : accuracy, f1, neg_log_loss, roc_auc  # clf cv default stratified k 방식
            model=GridSearchCV(model,param_grid=param_grid,scoring='roc_auc',return_train_score=True) # cv=2 최소, default 5
        elif (est =='reg'): # reg cv default k 방식
            model=GridSearchCV(model,param_grid=param_grid,scoring='neg_mean_squared_error',return_train_score=True) # cv=2 최소, default 5
        if (model_name in early_stopping_modellist):
            model.fit(X_train, y_train,early_stopping_rounds=20,eval_metric="logloss",eval_set=[(eval_X,eval_y)],verbose=False)
        else:
            model.fit(X_train, y_train) 
        best_params = model.best_params_
        print(f'GridSearchCV 최적 파라미터:{ best_params}')
        if (plot ==1):
            plot_grid_curve(model, param_grid, model_name, time_name)
        # metric
        model = model.best_estimator_
        metrics_dict = metricPlot(model,model_name,time_name,start_model_time,X_train,y_train,X_test,y_test)
        return model, param_grid, best_params, metrics_dict
   
    else: # grid = 1, 4
        if (model_name in early_stopping_modellist):
            model.fit(X_train, y_train,early_stopping_rounds=20,eval_metric="logloss",eval_set=[(eval_X,eval_y)],verbose=False)
        else:
            model.fit(X_train, y_train)
        metrics_dict = metricPlot(model,model_name,time_name,start_model_time,X_train,y_train,X_test,y_test)
        return model, metrics_dict      

### 3-1 trainMetricSave 실제 실행 함수 ## 3-2 get_model_train_eval 호출

In [29]:
def trainMetricSave(model_list,X_train,X_test,y_train,y_test,eval_X=None,eval_y=None):
    time_name = datetime.now().strftime("%Y%m%d-%H%M%S")
    params_dict={}
    for model in model_list:
        start_model_time = time.time()
        model_name = model.__class__.__name__
        param_grid=None
        best_params=None
        if(grid == 1 or grid == 4): # GridSerchCV 없이 초기 2. ~ 3. model 훈련, metric 리턴받아 처리 
            model, metrics_dict = \
                get_model_train_eval(model,model_name,time_name,start_model_time,X_train,X_test,y_train,y_test,eval_X,eval_y)
        elif(grid == 2): # RandomSearchCV 수행 ==> 2. data split ~ 3. model 훈련, metric 리턴 ####
            model, best_params, metrics_dict =\
                get_model_train_eval(model,model_name,time_name,start_model_time,X_train,X_test,y_train,y_test,eval_X,eval_y)
        elif(grid == 3): # GridSerchCV, RandomSearchCV 수행 ==> 2. data split ~ 3. model 훈련, metric 리턴 ####
            model, param_grid, best_params, metrics_dict =\
                get_model_train_eval(model,model_name,time_name,start_model_time,X_train,X_test,y_train,y_test,eval_X,eval_y)
        ## metrics json 저장 
        metrics_total_dict = {}
        spend_model_time=(time.time() - start_model_time)
        metrics_total_dict['Experiment_date_time'] = time_name
        metrics_total_dict['data_name'] = data_name
        metrics_total_dict['model_name'] = model_name
        if(param_grid):        
            metrics_total_dict['param_grid'] = param_grid 
        if(best_params):        
            metrics_total_dict['best_params'] = best_params
        metrics_total_dict['model_params'] = model.get_params()
        metrics_total_dict['metrics'] =metrics_dict
        metrics_total_dict['Execution_time'] = spend_model_time
        with open(f'./json/{data_name}_{model_name}_{time_name}.json', 'w', encoding='UTF-8-sig') as f:
            json.dump(metrics_total_dict, f, indent=4)            
        print(f'{model_name} 수행시간 : {int(spend_model_time//60)}분 {spend_model_time%60:.2f}초\n')  
        if(best_params):
            params = 'random_state=1'
            for key, value in best_params.items():
                params = params + ',' + str(key) + '=' + str(value)
    #             print(f'{model_name} params : {params}\n')
            params_dict[f'{model_name}']=params
    print(f'best params : {params_dict}')
    spend_time=(time.time() - start_time)
    print(f'전체수행시간 : {int(spend_time//60)}분 {spend_time%60:.2f}초\n')  

## 1. data preprocessing, split

In [30]:
# 1.필요시 전처리 리턴 => 2. X,y(target)분리 => 3. 필요시 scaling => 4. train,test로 split
def get_train_test_dataset(df=None):
    y_target = df['target']
    X_features = df.drop(['target'], axis=1,inplace=False)
    
    ##  scaler 필요시
    columns=X_features.columns
#     scaler = MinMaxScaler()
#     scaler = StandardScaler()
#     X_features = scaler.fit_transform(X_features) # nd.array 변환됨 df변환 필요
    X_features = np.log1p(X_features)
    ## 다항 
#     X_features = PolynomialFeatures(degree=2, include_bias=False).fit_transform(X_features)
#     X_features = pd.DataFrame(X_features) # polynomialFeatures 사용시 df변환
    X_features = pd.DataFrame(X_features, columns=columns) # df 변환
    if(yLog == True):
        y_target = np.log1p(y_target)

    # oneHotencoder
    X_features = pd.get_dummies(X_features)
    
    ## train_test_split( )으로 학습과 테스트 데이터 분할. 
    # 분류에서 target 불균형시 stratify=y_target으로 Stratified 기반 분할, # 회귀에서는 stratify사용시 error
    X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size=0.2, random_state=156)
    # xgb, lgbm early_stopping_rounds 사용시 ==> test 데이터 eval_set 으로 다시 분리 
    X_train, eval_X, y_train, eval_y = train_test_split(X_train, y_train, test_size=0.2, random_state=256)
    
#     return X_train, X_test, y_train, y_test
    return X_train, X_test, y_train, y_test, eval_X, eval_y
#     return scaler, p_degree, X_train, X_test, y_train, y_test
#     return scaler, p_degree, X_train, X_test, y_train, y_test, eval_X, eval_y

## RUN  
### data 선택 (1 get_train_test_dataset 호출), model 선택후 run  ##  3-1 trainMetricSave 호출

In [ ]:
# 수행 시간 측정을 위함. 시작 시간 설정.
start_time = time.time()

data_name = 'project_reg_sampling_1_1_re'

est = "clf"    # 맞는 df 선택 # pd.get_dummies() oHe 여부, target scaling여부
# est = "reg"    # 맞는 df 선택 # pd.get_dummies() oHe 여부, target scaling여부

# target scaler(회귀)했다면(yLog) metric할 때 is_expm1
# yLog=False
yLog=True

grid = 1      # 초기모델링    # 초기 model_list
# grid = 2     # RandomSearchCV 수행    # params 조정
# grid = 3     # GridSearchCV 수행     # params 조정
# grid = 4    # best params 적용  # 해당 model_list

plot = 0    # plot  disable 수행시간때문에
# plot = 1  # plot enable 

def run():    
    ## data 데이터 선택 ########################################
#     df = dataset()
#     df = bike_df
#     df = house_df_ohe
#     df = pima_df
    # project
    df = pd.read_csv('./data/NHIS_total_model.csv')
#     df = pd.read_csv('./data/NHIS_model_1.csv')        # clf
#     df = pd.read_csv('./data/NHIS_model_clf_PI_1.csv')        # clf
#     df = pd.read_csv('./data/NHIS_model_reg_2.csv')  # reg
#     df = pd.read_csv('./data/NHIS_model_reg_4_corr.csv')  # reg
#     df = pd.read_csv('./data/NHIS_model_reg_4_sampling_1_1.csv')  # reg
#     df = df.iloc[:1000, :]
#     df = sklearn.utils.shuffle(df)

    # clf
#     target_count = df[df['target'] == 1].shape[0]
#     target_ex_count = df[df['target'] != 1].shape[0]
#     print(f"target   count : {target_count} ({ round( target_count/df.shape[0], 4)*100 })%")
#     print(f"target외 count : {target_ex_count} ({ round( target_ex_count/df.shape[0], 4)*100 })%")

    ## 1. 데이터 전처리(preprocessing), 분리(split) ##########################################
#     X_train, X_test, y_train, y_test = get_train_test_dataset(df)
#     scaler, p_degree, X_train, X_test, y_train, y_test = get_train_test_dataset(df)
    X_train, X_test, y_train, y_test, eval_X, eval_y = get_train_test_dataset(df)
    
    ## model 선택 ##################################### 'squared_loss’, 'huber’,'epsilon_insensitive’,squared_epsilon_insensitive’
    if (est=='clf'):
        model_list= [
                 LogisticRegression(random_state=1, n_jobs=-1),
                 LinearSVC(random_state=1, loss='hinge'),
#                  SGDClassifier(random_state=1,n_jobs=-1,loss='modified_huber', early_stopping=True), # xgb,lgbm:fit조기중단가능
#                  DecisionTreeClassifier(random_state=1),
#                  RandomForestClassifier(random_state=1,n_jobs=-1),
#                  ExtraTreeRegressor(random_state=1,n_jobs=-1)
#                  GradientBoostingClassifier(random_state=1,validation_fraction=0.1,n_iter_no_change=20,subsample=0.25),
#                  XGBClassifier(random_state=1,n_jobs=-1,silent=True,device='gpu'),
#                  LGBMClassifier(random_state=1,n_jobs=-1),
            
                ]
    elif (est=='reg'): # LinearSVR, GBM 수행시간 느림
        model_list = [
                      LinearRegression(n_jobs=-1), 
#                       LinearSVR(random_state=1,verbose=0), # 수행시간 느림 metric 65분
#                       SGDRegressor(random_state=1,early_stopping=True), # yLog=True  error
                      Ridge(random_state=1),
                      Lasso(random_state=1),
                      ElasticNet(random_state=1),
                      DecisionTreeRegressor(random_state=1),
                      RandomForestRegressor(random_state=1,n_jobs=-1),
                      ExtraTreeRegressor(random_state=1,n_jobs=-1),
#                       GradientBoostingRegressor(random_state=1,validation_fraction=0.1,n_iter_no_change=20,subsample=0.25), # 
                      XGBRegressor(random_state=1, n_jobs=-1, silent=True,device='gpu'),
                      LGBMRegressor(random_state=1, n_jobs=-1),
                      CatBoostClassifier(random_state=1, n_jobs=-1)
                      
                     ] 
    
#     model_list= [
#         LinearRegression(n_jobs=-1,fit_intercept=True), # random_state 없음
#         LinearSVR(random_state=1, C=1,max_iter=1000),
#         Ridge(random_state=1, alpha=10,solver='auto'),
#         Lasso(random_state=1, alpha=100,max_iter=1000),
#         ElasticNet(random_state=1, alpha=0.1,l1_ratio=0.9),
#         DecisionTreeRegressor(random_state=1, max_depth=10,min_samples_leaf=10,min_samples_split=5),
#         RandomForestRegressor(n_jobs=-1, random_state=1, max_depth=20,min_samples_leaf=5,min_samples_split=5),
#         GradientBoostingRegressor(random_state=1, max_depth=10,min_samples_leaf=20,min_samples_split=10),
#         XGBRegressor(n_jobs=-1, random_state=1, silent=True, max_depth=5,min_child_weight=0.1,subsample=0.8),
#         LGBMRegressor( n_jobs=-1,random_state=1,max_depth=64,min_child_samples=20,num_leaves=64,subsample=0.6)
#     ] 

    ##  2. ~ 3. 훈련, metric  ###################################################
    trainMetricSave(model_list, X_train, X_test, y_train, y_test, eval_X, eval_y)
    
run()

In [ ]:
model_list= [
    LogisticRegression(random_state=1,n_jobs=-1,C=0.1,l1_ratio=0.1,max_iter=100,solver='lbfgs'),
    KNeighborsClassifier(n_jobs=-1,n_neighbors=10,p=1),
    LinearSVC(random_state=1, loss='hinge'),
    SGDClassifier(random_state=1,n_jobs=-1,alpha=0.0001,epsilon=0.1,eta0=0.0,l1_ratio=0.01,max_iter=100), 
    DecisionTreeClassifier(random_state=1,max_depth=5,max_features=8,min_samples_leaf=20,min_samples_split=10),
    RandomForestClassifier(random_state=1,n_jobs=-1,max_depth=5,max_features=5,min_samples_leaf=15,min_samples_split=10),
    GradientBoostingClassifier(random_state=1,max_depth=10,max_features=5,min_samples_leaf=10,min_samples_split=10),
    XGBClassifier(random_state=1,n_jobs=-1,silent=True,colsample_bytree=0.5,max_depth=15,min_child_weight=1,reg_alpha=0,reg_lambda=0.5,subsample=1),
    LGBMClassifier(random_state=1,n_jobs=-1,colsample_bytree=1,max_depth=128,min_child_samples=5,num_leaves=64,reg_alpha=0,reg_lambda=0.5,subsample=0.8)
]

model_list= [
    LinearRegression(n_jobs=-1,fit_intercept=True), # random_state 없음
    LinearSVR(random_state=1, C=1,max_iter=1000),
    SGDRegressor(random_state=1),
    Ridge(random_state=1, alpha=10,solver='auto'),
    Lasso(random_state=1, alpha=100,max_iter=1000),
    ElasticNet(random_state=1, alpha=0.1,l1_ratio=0.9),
    DecisionTreeRegressor(random_state=1, max_depth=10,min_samples_leaf=10,min_samples_split=5),
    RandomForestRegressor(n_jobs=-1, random_state=1, max_depth=20,min_samples_leaf=5,min_samples_split=5),
    GradientBoostingRegressor(random_state=1, max_depth=10,min_samples_leaf=20,min_samples_split=10),
    XGBRegressor(n_jobs=-1, random_state=1, silent=True, max_depth=5,min_child_weight=0.1,subsample=0.8),
    LGBMRegressor( n_jobs=-1,random_state=1,max_depth=64,min_child_samples=20,num_leaves=64,subsample=0.6)
] 

## 개별 model별 
### Clf

In [ ]:
df.columns

In [16]:
NGBClassifier().get_params()

{'Base__ccp_alpha': 0.0,
 'Base__criterion': 'friedman_mse',
 'Base__max_depth': 3,
 'Base__max_features': None,
 'Base__max_leaf_nodes': None,
 'Base__min_impurity_decrease': 0.0,
 'Base__min_impurity_split': None,
 'Base__min_samples_leaf': 1,
 'Base__min_samples_split': 2,
 'Base__min_weight_fraction_leaf': 0.0,
 'Base__presort': 'deprecated',
 'Base__random_state': None,
 'Base__splitter': 'best',
 'Base': DecisionTreeRegressor(criterion='friedman_mse', max_depth=3),
 'Dist': ngboost.distns.categorical.k_categorical.<locals>.Categorical,
 'Score': ngboost.scores.LogScore,
 'col_sample': 1.0,
 'learning_rate': 0.01,
 'minibatch_frac': 1.0,
 'n_estimators': 500,
 'natural_gradient': True,
 'random_state': RandomState(MT19937) at 0x1ED77B8C8C8,
 'tol': 0.0001,
 'verbose': True,
 'verbose_eval': 100}

In [31]:
start_time = time.time()

# data_name = "pima"
data_name = "project_clf_104M_ng"

grid = 1      # 초기모델링    # 초기 model_list, # target scaler했다면 is_expm1=False, True 확인
# grid = 2     # RandomSearchCV 수행    # params 조정
# grid = 3     # GridSearchCV 수행     # params 조정
# grid = 4    # best params 적용  # 해당 model_list

# plot = 0    # plot  disable 수행시간때문에
plot = 1  # plot enable 

est = "clf" 

# df = pd.read_csv('data/NHIS_total_model.csv')
# df = pima_df
df = pd.read_csv('data/NHIS_model_1.csv')
# df = df.iloc[:200000, :]
# df['target'] = df['식전혈당(공복혈당)'].apply(lambda x: 1 if x >= 126 else 0 )
# df.drop('식전혈당(공복혈당)', axis=1, inplace=True)    

target_count = df[df['target'] == 1].shape[0]
target_ex_count = df[df['target'] != 1].shape[0]
print(f"target   count : {target_count} ({ round( target_count/df.shape[0], 4)*100 })%")
print(f"target외 count : {target_ex_count} ({ round( target_ex_count/df.shape[0], 4)*100 })%")

y_target = df['target']
X_features = df.drop(['target', '기준년도', '시도코드'], axis=1)
# X_features = X = df[['Glucose','BloodPressure','BMI','Age']]

## scaler시
# columns = X_features.columns
# scaler = MinMaxScaler()
# scaler = StandardScaler()
# X_features = scaler.fit_transform(X_features)
# X_features = np.log1p(X_features)
# X_features = pd.DataFrame(X_features, columns=columns)

## 데이터 학습과 테스트 데이터 분할 
X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size=0.2, random_state=156,stratify=y_target)
# xgb,lgbm,sgb early_stopping_rounds 사용시 ==> test 데이터 eval_set 으로 다시 분리  # train, test 분리 0.3 필요
X_test, eval_X, y_test, eval_y = train_test_split(X_test, y_test, test_size=0.2, random_state=256, stratify=y_test)

## 모델 선택
# model = LogisticRegression(random_state=1, n_jobs=-1)  
# model = SVC(random_state=1, kernel="linear", C=1, probability=True) # StandardScaler  # 
# model = SVC(random_state=1,probability=True, kernel='poly', degree=3, coef0=1, C=5) # StandardScaler
# model = SVC(random_state=1,probability=True, kernel='rbf', gamma=5, C=0.001) # StandardScaler
# model = LinearSVC(random_state=1, loss='hinge')
# model = KNeighborsClassifier(n_jobs=-1)
# model = SGDClassifier(random_state=1,n_jobs=-1)
# model =  SGDClassifier(random_state=1,n_jobs=-1, loss='log')  # log => logit + SGD
# model= GaussianNB()
# model = tree.DecisionTreeClassifier(random_state=1)
# model = DecisionTreeClassifier(random_state=1)
# model = RandomForestClassifier(random_state=1,n_jobs=-1)
# model = GradientBoostingClassifier(random_state=1),
# model = XGBClassifier(random_state=1,colsample_bytree=0.5734329430545178,learning_rate=0.03251204127577167,
#                     max_depth=22,min_child_weight=0.002426065522925542,n_estimators=429,subsample=0.9991897949519826,
#                     silent=True,n_jobs=-1, device='gpu')
# model = LGBMClassifier(random_state=1,colsample_bytree=0.5,learning_rate=0.3,min_child_samples=300,min_child_weight=900,
#                        n_estimators=800,num_leaves=200,reg_alpha=0.057,reg_lambda=0.021,subsample=0.4)
# model = CatBoostClassifier(verbose=0, n_estimators=1000)
model = NGBClassifier(random_state=1) # 정확도: 0.7182, 정밀도: 0.6948, 재현율: 0.7782, f1_score: 0.7341, auc_score:0.7919 # 36분 # plot하지 말것 
  
## 모델 훈련 및 성능평가(metric)
time_name = datetime.now().strftime("%Y%m%d-%H%M%S")
model_name = model.__class__.__name__
model.fit(X_train, y_train)
# model.fit(X_train, y_train, early_stopping_rounds=10, eval_metric="logloss", eval_set=[(eval_X, eval_y)], verbose=True)
pred = model.predict(X_test)
if model_name in ['LinearSVC']:
    pred_proba = model._predict_proba_lr(X_test) 
else:
    pred_proba = model.predict_proba(X_test) 
metrics_dict = get_clf_eval(y_test, pred, pred_proba[:,1])
spend_metric_time=(time.time() - start_time)
print(f'{model_name} metric 수행시간 : {int(spend_metric_time//60)}분 {spend_metric_time%60:.2f}초\n')  

## model 저장
# pickle.dump(model, open(f'./modeling/pima_{model_name}_{time_name}.sav', 'wb'))

## plot
Plot_learning_curve(model,model_name,time_name,X_train,y_train,n_jobs=-1,train_sizes=np.linspace(.1,1.0,5))
precision_recall_curve_plot(y_test, pred_proba[:,1], model_name, time_name )
roc_curve_plot(y_test, pred_proba[:,1], model_name, time_name)
if model_name in ['LogisticRegression','SGDClassifier','LinearSVC']:
    Plot_coef_logit(model, X_train.columns, model_name, time_name) # 회귀계수 
else : 
    Plot_feature_importance(model, X_train.columns, model_name, time_name) # 회귀트리 # lr x
results = permutation_importance(model, X_test, y_test, scoring='f1')
importance = results.importances_mean
Plot_permutation_importance(importance, X_test.columns, model_name, time_name)
    
## metrics json 저장 
metrics_total_dict = {}
spend_time=(time.time() - start_time)
metrics_total_dict['Experiment_date_time'] = time_name
metrics_total_dict['data_name'] = data_name
metrics_total_dict['model_name'] = model_name
metrics_total_dict['model_params'] = model.get_params()
metrics_total_dict['metrics'] =metrics_dict
metrics_total_dict['Execution_time'] = spend_time
# with open(f'./json1/{data_name}_{model_name}_{time_name}.json', 'w', encoding='UTF-8-sig') as f:
with open(f'./json1/{data_name}_{model_name}_{time_name}.json', 'w', encoding='UTF-8-sig') as f:
    json.dump(metrics_total_dict, f, indent=4)            
print(f'{model_name} 전체 수행시간 : {int(spend_time//60)}분 {spend_time%60:.2f}초\n')  


target   count : 521858 (50.0)%
target외 count : 521754 (50.0)%
[iter 0] loss=0.6931 val_loss=0.0000 scale=2.0000 norm=4.0000
[iter 100] loss=0.5857 val_loss=0.0000 scale=2.0000 norm=3.7418
[iter 200] loss=0.5649 val_loss=0.0000 scale=2.0000 norm=3.7782
[iter 300] loss=0.5570 val_loss=0.0000 scale=1.0000 norm=1.9037
[iter 400] loss=0.5524 val_loss=0.0000 scale=1.0000 norm=1.9166
 정확도: 0.7182, 정밀도: 0.6948, 재현율: 0.7782, f1_score: 0.7341, auc_score:0.7919
NGBClassifier metric 수행시간 : 36분 26.17초



KeyboardInterrupt: 

### Reg

In [17]:
NGBRegressor().get_params()

{'Base__ccp_alpha': 0.0,
 'Base__criterion': 'friedman_mse',
 'Base__max_depth': 3,
 'Base__max_features': None,
 'Base__max_leaf_nodes': None,
 'Base__min_impurity_decrease': 0.0,
 'Base__min_impurity_split': None,
 'Base__min_samples_leaf': 1,
 'Base__min_samples_split': 2,
 'Base__min_weight_fraction_leaf': 0.0,
 'Base__presort': 'deprecated',
 'Base__random_state': None,
 'Base__splitter': 'best',
 'Base': DecisionTreeRegressor(criterion='friedman_mse', max_depth=3),
 'Dist': ngboost.distns.normal.Normal,
 'Score': ngboost.scores.LogScore,
 'col_sample': 1.0,
 'learning_rate': 0.01,
 'minibatch_frac': 1.0,
 'n_estimators': 500,
 'natural_gradient': True,
 'random_state': RandomState(MT19937) at 0x1ED77B8C8C8,
 'tol': 0.0001,
 'verbose': True,
 'verbose_eval': 100}

In [32]:
start_time = time.time()

# data_name = "bostonHouse"
# data_name = "bike"
# data_name = "project"
data_name = "project_reg_4_sample_1_1"

grid = 1      # 초기모델링    # 초기 model_list, # target scaler했다면 is_expm1=False, True 확인
# grid = 2     # RandomSearchCV 수행    # params 조정
# grid = 3     # GridSearchCV 수행     # params 조정
# grid = 4    # best params 적용  # 해당 model_list

# plot = 0    # plot  disable 수행시간때문에
# plot = 1  # plot enable 

est = "reg" 
yLog = 1
# yLog = False
 
## project data
# df = pd.read_csv('data/NHIS_total_model.csv')
# df = pd.read_csv('./data/NHIS_model_reg_1.csv')  # reg
df = pd.read_csv('./data/NHIS_model_reg_4_sampling_1_1.csv')  # reg
# df = df.iloc[:1000000, :]
# df = sklearn.utils.shuffle(df)

y_target = df['target']
X_features = df.drop(['target'], axis=1,inplace=False)

y_target = np.log1p(y_target)

##  scaler 필요시
columns=X_features.columns
# scaler = MinMaxScaler()
scaler = StandardScaler()
X_features = scaler.fit_transform(X_features) # nd.array 변환됨 df변환 필요
# X_features = np.log1p(X_features)
## 다항 
# X_features = PolynomialFeatures(degree=2, include_bias=False).fit_transform(X_features)
# X_features = pd.DataFrame(X_features) # polynomialFeatures 사용시 df 변환
X_features = pd.DataFrame(X_features, columns=columns) # df 변환
X_features = pd.get_dummies(X_features) # oHe

## 데이터 학습과 테스트 데이터 분할  # stratify 회귀에서는 삭제
X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size=0.2, random_state=156)
# xgb,lgbm,sgb early_stopping_rounds 사용시 ==> test 데이터 eval_set 으로 다시 분리  # train, test 분리 0.3 필요
X_test, eval_X, y_test, eval_y = train_test_split(X_test, y_test, test_size=0.2, random_state=256)


## 아래 params에서 n_jobs=-1이 있는 곳은 그대로 추가 # xgb는 silent=True까지 추가
# model = LinearRegression(n_jobs=-1,fit_intercept=True)
# model = LinearSVR(random_state=1, C=1,max_iter=100)
# model = SGDRegressor(random_state=1)
# model = SVR(kernel="linear", C=1) # StandardScaler  # 
# model = SVR(kernel='poly', degree=2, epsilon=0.1, C=100) # StandardScaler
# model = SVR(kernel='rbf', gamma=5, C=0.001) # StandardScaler
# model = Ridge(random_state=1, alpha=10,solver='auto')
# model = Lasso(random_state=1, alpha=0.1,max_iter=100)
# model = ElasticNet(random_state=1, alpha=0.1,l1_ratio=0.9)

# model = DecisionTreeRegressor(random_state=1, max_depth=13,min_samples_leaf=5,min_samples_split=5)
# model = RandomForestRegressor(n_jobs=-1, random_state=1, max_depth=20,min_samples_leaf=5,min_samples_split=5)
# model = GradientBoostingRegressor(random_state=1, max_depth=13,min_samples_leaf=10,min_samples_split=50)
# model = XGBRegressor(n_jobs=-1, random_state=1, silent=True, max_depth=20,min_child_weight=7,subsample=0.6)
# model = LGBMRegressor( n_jobs=-1,random_state=1,max_depth=64,min_child_samples=20,num_leaves=128,subsample=0.6)
# model = ExtraTreeRegressor(random_state=1)
# model = CatBoostRegressor( n_jobs=-1,random_state=1)
model = NGBRegressor(random_state=1) # MAE: 27.985, MSE: 1872.801, RMSE: 43.276, RMSLE: 0.273, R2: 0.187 # 50분
  
## 모델 훈련 및 성능평가(metric)
time_name = datetime.now().strftime("%Y%m%d-%H%M%S")
model_name = model.__class__.__name__
model.fit(X_train, y_train)
# model.fit(X_train, y_train, early_stopping_rounds=10, eval_metric="logloss", eval_set=[(eval_X, eval_y)], verbose=True)
pred = model.predict(X_test)
if yLog : #is_expm1 <= targetlog변환시 복원
    y_test = np.expm1(y_test)
    pred = np.expm1(pred)  
metrics_dict = get_reg_eval(y_test, pred)
spend_metric_time=(time.time() - start_time)
print(f'{model_name} metric 수행시간 : {int(spend_metric_time//60)}분 {spend_metric_time%60:.2f}초\n')  

## model 저장
# pickle.dump(model, open(f'./modeling/{model_name}_{time_name}.sav', 'wb'))

## plot
# Plot_learning_curve(model,model_name,time_name,X_train,y_train,n_jobs=-1,train_sizes=np.linspace(.1,1.0,5))
# if model_name in ['LinearRegression','SGDRegressor','LinearSVR']:
#     Plot_coef_logit(model, X_train.columns, model_name, time_name) # 회귀계수 
# else : 
#     Plot_feature_importance(model, X_train.columns, model_name, time_name) # 회귀트리 # lr x
# results = permutation_importance(model, X_test, y_test, scoring='neg_mean_absolute_error')  # r2
# importance = results.importances_mean
# Plot_permutation_importance(importance, X_test.columns, model_name, time_name)
    
## metrics json 저장  #  NGBRegressor error : Object of type DecisionTreeRegressor is not JSON serializable
metrics_total_dict = {}
spend_time=(time.time() - start_time)
metrics_total_dict['data_name'] = data_name
metrics_total_dict['model_name'] = model_name
model_params = model.get_params() # dict
metrics_total_dict['model_params'] = model_params
# metrics_total_dict['scaler'] = scaler
# metrics_total_dict['p_degree'] = p_degree
metrics_total_dict['metrics'] =metrics_dict
metrics_total_dict['Execution_time'] = spend_time
# with open(f'./json1/{data_name}_{model_name}_{time_name}.json', 'w', encoding='UTF-8-sig') as f:
with open(f'./json1/{data_name}_{model_name}_{time_name}.json', 'w', encoding='UTF-8-sig') as f:
    json.dump(metrics_total_dict, f, indent=4)            
print(f'수행시간 : {int(spend_time//60)}분 {spend_time%60:.2f}초')        

[iter 0] loss=0.2746 val_loss=0.0000 scale=1.0000 norm=0.5614
[iter 100] loss=0.1625 val_loss=0.0000 scale=2.0000 norm=1.0922
[iter 200] loss=0.1161 val_loss=0.0000 scale=1.0000 norm=0.5640
[iter 300] loss=0.0997 val_loss=0.0000 scale=2.0000 norm=1.1425
[iter 400] loss=0.0907 val_loss=0.0000 scale=1.0000 norm=0.5749
MAE: 27.985, MSE: 1872.801, RMSE: 43.276, RMSLE: 0.273, R2: 0.187

NGBRegressor metric 수행시간 : 50분 6.58초



TypeError: Object of type DecisionTreeRegressor is not JSON serializable

In [ ]:
# pima diabetes
pima_df = pd.read_csv('data/diabetes.csv')
df=pima_df
#0 제거 후 nan 으로 변환 # np.nan의 type : float
col=['Glucose', 'BloodPressure','SkinThickness','Insulin','BMI' ]
for i in col:
    df[i].replace(0, np.nan, inplace= True)
    # median 중앙값 찾기
def median_target(var):   
    temp = df[df[var].notnull()]
    temp = temp[[var, 'Outcome']].groupby(['Outcome'])[[var]].median().reset_index()
    return temp
# o값 대체 NaN 채워넣기 
df.loc[(df['Outcome'] == 0 ) & (df['Glucose'].isnull()), 'Glucose'] = median_target('Glucose')[ median_target('Glucose')['Outcome']==0]['Glucose'].values[0]
df.loc[(df['Outcome'] == 1 ) & (df['Glucose'].isnull()), 'Glucose'] = median_target('Glucose')[ median_target('Glucose')['Outcome']==1]['Glucose'].values[0]
df.loc[(df['Outcome'] == 0 ) & (df['BloodPressure'].isnull()), 'BloodPressure'] = median_target('BloodPressure')[ median_target('BloodPressure')['Outcome']==0]['BloodPressure'].values[0]
df.loc[(df['Outcome'] == 1 ) & (df['BloodPressure'].isnull()), 'BloodPressure'] = median_target('BloodPressure')[ median_target('BloodPressure')['Outcome']==1]['BloodPressure'].values[0]
df.loc[(df['Outcome'] == 0 ) & (df['SkinThickness'].isnull()), 'SkinThickness'] = median_target('SkinThickness')[ median_target('SkinThickness')['Outcome']==0]['SkinThickness'].values[0]
df.loc[(df['Outcome'] == 1 ) & (df['SkinThickness'].isnull()), 'SkinThickness'] = median_target('SkinThickness')[ median_target('SkinThickness')['Outcome']==1]['SkinThickness'].values[0]
df.loc[(df['Outcome'] == 0 ) & (df['Insulin'].isnull()), 'Insulin'] =  median_target('Insulin')[ median_target('Insulin')['Outcome']==0]['Insulin'].values[0]
df.loc[(df['Outcome'] == 1 ) & (df['Insulin'].isnull()), 'Insulin'] = median_target('Insulin')[ median_target('Insulin')['Outcome']==1]['Insulin'].values[0]
df.loc[(df['Outcome'] == 0 ) & (df['BMI'].isnull()), 'BMI'] = median_target('BMI')[ median_target('BMI')['Outcome']==0]['BMI'].values[0]
df.loc[(df['Outcome'] == 1 ) & (df['BMI'].isnull()), 'BMI'] = median_target('BMI')[ median_target('BMI')['Outcome']==1]['BMI'].values[0]
# df.isnull().sum()
df['target'] = df['Outcome']
df.drop('Outcome', axis=1, inplace=True)
pima_df=df
# pima_df.info()

In [ ]:
0. data_name, df 선택 ==> target 분리, 전처리영역필요여부
1. 초기모델은 grid = 1 선택, 전체 model_list for문, 과적합 learning_curve plot 확인
    model_list,  Plot learning_curve
    
2. GridSearchCV 사용 grid = 2 선택, 전체 model_list for문, parameter별 영향력 확인 grid_curve
    model_list,  plot grid_curve
    ==> 1항에서 parameter tunning 반복가능
    
3. grid = 3 best param model 선택, 과적합 learning_curve plot 확인
    model_list, Plot learning_curve
    
4. model과 최적 param 찾았다면, model
    model, Plot (coeff_, feature_importance_ )
    
5. feature engineering : scaler, p_degree(다항식), 
    scaler log1p 변환시 is_expm1=True
    1. 3.항 실행
6. xgb, lgbm의 early기능 사용시 eval_X, eval_y  

In [ ]:
# with open(f'./static/json/{model_name}_{time_name}.json', 'r', encoding='UTF-8-sig') as json_file:
    #     data = json.load(json_file)
    #     print(data)
def dictParse(data):
    time_name = data['Experiment_date_time']
    data_name= data['data_name']
    model_name= data['model_name']
    model_params = data['model_params']
    if 'scaler' in data.keys():
        scaler = data['scaler']
    if 'p_degree' in data.keys():
        p_degree = data['p_degree']
    mae_v = data['metrics']['MAE']
    mse_v = data['metrics']['MSE']
    rmse_v = data['metrics']['RMSE']
    rmsle_v = data['metrics']['RMSLE']
    Execution_time = data['Execution_time'] 
    return data_name, model_name, model_params, rmsle_v, rmse_v, mse_v, Execution_time
metrics_total_dict = run()
data_name, model_name, model_params, rmsle_v, rmse_v, mse_v, Execution_time = dictParse(metrics_total_dict)
# model_n, scaler, p_degree, rmsle_v, rmse_v, mse_v, Execution_time = jsonParse(metrics_total_dict)
print(f'Experiment_date_time:{time_name}\n data_name : {data_name}\n model_name : {model_name}\n model_params : {model_params}\n MAE {mae_v}\n MSE {mse_v}\n RMSE {rmse_v}\n  RMSLE {rmsle_v}\n Execution_time {Execution_time}')
# print(f'model name : {model_n}\n model_params : {model_parma}\n scaler {scaler}\n p_degree {p_degree}\n RMSLE {rmsle_v}\n RMSE {rmse_v}\n MAE {mae_v}\n Execution_time {Execution_time}')    

In [ ]:
## data관련

def dataset():
    from sklearn.datasets import load_iris
    from sklearn.datasets import load_breast_cancer # xgb 유방암, voting사용
    from sklearn.datasets import load_diabetes  # pima아니고 
    from sklearn.datasets import load_boston # 회귀
   
    # 보스턴 데이터 세트 로드
    boston = load_boston()
    df = pd.DataFrame(boston.data, columns = boston.feature_names)
    df['PRICE'] = boston.target
#     print(df.info()) # 506
    return df
# dataset()

bike_df = pd.read_csv('data/bike_train.csv')
# 문자열을 datetime 타입으로 변경. 
bike_df['datetime'] = bike_df.datetime.apply(pd.to_datetime)
# datetime 타입에서 년, 월, 일, 시간 추출
bike_df['year'] = bike_df.datetime.apply(lambda x : x.year)
bike_df['month'] = bike_df.datetime.apply(lambda x : x.month)
bike_df['day'] = bike_df.datetime.apply(lambda x : x.day)
bike_df['hour'] = bike_df.datetime.apply(lambda x: x.hour)
drop_columns = ['datetime','casual','registered'] # registered와 count 거의 동일해서
bike_df.drop(drop_columns, axis=1,inplace=True)
# print(bike_df.info()) # 10886
# df = bike_df

# AmesHouse
house_df = pd.read_csv('data/house_price.csv')
# Null 이 너무 많은 컬럼들과 불필요한 컬럼 삭제
house_df.drop(['Id','PoolQC' , 'MiscFeature', 'Alley', 'Fence','FireplaceQu'], axis=1 , inplace=True)
# Drop 하지 않는 숫자형 Null컬럼들은 평균값으로 대체
house_df.fillna(house_df.mean(),inplace=True)
null_column_count = house_df.isnull().sum()[house_df.isnull().sum() > 0]
# print('## Null 피처의 Type :\n', house_df.dtypes[null_column_count.index])
# 문자형(범주형) feature one-hot 인코딩으로 shape 변환됨 (1460, 75) ==> (1460, 271) 
house_df_ohe = pd.get_dummies(house_df)
null_column_count = house_df_ohe.isnull().sum()[house_df_ohe.isnull().sum() > 0]
# print('## Null 피처의 Type :\n', house_df_ohe.dtypes[null_column_count.index])

# AmesHouse scaler2 ==> X_features skew큰것 log1p변환
from scipy.stats import skew
# object가 아닌 숫자형 피쳐의 컬럼 index 객체 추출.
features_index = house_df.dtypes[house_df.dtypes != 'object'].index
# house_df에 컬럼 index를 [ ]로 입력하면 해당하는 컬럼 데이터 셋 반환. apply lambda로 skew( )호출 
skew_features = house_df[features_index].apply(lambda x : skew(x))
# skew 정도가 1 이상인 컬럼들만 추출. 
skew_features_top = skew_features[skew_features > 1]
# print(skew_features_top.sort_values(ascending=False))
house_df[skew_features_top.index] = np.log1p(house_df[skew_features_top.index])
house_df_ohe = pd.get_dummies(house_df)

In [ ]:
def get_train_test_dataset(df=None):

    ## target과 train 데이터 분리
    ## clf
    # pima
#     y_target = df['Outcome']
#     X_features = df.drop('Outcome', axis=1)
    # customer(bank)
#     y_target = df['TARGET']
#     X_features = df.drop('TARGET', axis=1)    

    # credit
#     y_target = df['Class']
#     X_features = df.drop('Class', axis=1)    
#     scaler = StandardScaler()
#     amount_n = scaler.fit_transform(X_features['Amount'].values.reshape(-1, 1))
#     amount_n = np.log1p(X_features['Amount'])
#     X_features.insert(0, 'Amount_Scaled', amount_n)
#     X_features.drop(['Amount'], axis=1, inplace=True)

    ## reg
    #bostonHouse
#     y_target = df['PRICE']
#     X_features = df.drop(['PRICE'], axis=1,inplace=False)
    # bike
#     y_target = df['count']
#     X_features = df.drop(['count'],axis=1,inplace=False)
    # Ameshouse
#     y_target = house_df_ohe['SalePrice']
#     X_features = house_df_ohe.drop('SalePrice',axis=1, inplace=False)

### metric 함수 참고

In [ ]:
# cross_val_score이용한  5 Fold 셋으로 MSE 를 구한 뒤 이를 기반으로 다시  RMSE 구함. 
# scoring="neg_mean_squared_error" # GridSearchCV에서도 동일함
def get_model_cv_prediction(model, X_data, y_target):
    neg_mse_scores = cross_val_score(model, X_data, y_target, scoring="neg_mean_squared_error", cv = 5)
    rmse_scores  = np.sqrt(-1 * neg_mse_scores)
    avg_rmse = np.mean(rmse_scores)
    print('##### ',model.__class__.__name__ , ' #####')
    print(' 5 교차 검증의 평균 RMSE : {0:.3f} '.format(avg_rmse))

In [ ]:
def get_top_error_data(y_test, pred, n_tops = 5):
    # DataFrame에 컬럼들로 실제 대여횟수(count)와 예측 값을 서로 비교 할 수 있도록 생성. 
    result_df = pd.DataFrame(y_test.values, columns=['real_count'])
    result_df['predicted_count']= np.round(pred)
    result_df['diff'] = np.abs(result_df['real_count'] - result_df['predicted_count'])
    # 예측값과 실제값이 가장 큰 데이터 순으로 출력. 
    print(result_df.sort_values('diff', ascending=False)[:n_tops])
    
# get_top_error_data(y_test,pred,n_tops=5)

In [ ]:
# get_params()관련 LGBM기준 (XGB유사) # sklearn version별 api 다름
# max_depth log2(데이터수) # bostonHouse 506=>8, # bike 10886=>13 # AmesHouse 1460=>10 # project 797만=>22
# num_leaves  : 2^(max_depth) 작아야 함  최대 리프노드 갯수 = max_leaf_nodes(dt, rf)
# min_child_samples : Leaf node가 되기 위한 최소한 데이터 개체수  = min_samples_leaf  # min_samples_split (dt,rf 분리 최소 데이터)
# min_child_weight = min_child_leaf (dt, rf) *gbm은 없음    
# subsample (gbm,xgb,lgbm) 데이터 샘플링 비율 *lgbm은 subsample_for_bin 20000, subsample_freq 0
# colsample_bytree = max_features (dt, rf, gbm) 개별 트리를 학습할 때마다 무작위로 선택하는 feature 비율을 제어 (feature 많을 경우) * 
    # XGB : colsample_bylevel, colsample_bynode, colsample_bytree
# reg_alpha : L1 규제(제거) # reg_lambda : L2 규제(최소화) 
# 영혼까지 learning_rate 내리고,  n_estimators(rf,gbm,xgb,lgbm) 올리고
def get_params(model_name, data_name):  # param_grid 2개항목 이상일때만 plot_grid_curve 가능
    if (grid == 2) : # RandomSearch    속도느링 LogisticRegression  GradientBoostingClassifier
        # clf
        if model_name == 'LogisticRegression': # 속도느링 #"solver":['lbfgs','auto']
            param_grid = { 'C': stats.loguniform(1e-1, 1e0),'l1_ratio': stats.uniform(0, 1), "max_iter":stats.randint(100,500) }
        elif model_name == 'LinearSVC': # 'loss' ['hinge',''] # C 커지면 곡선 -> 직선 # tol(허용오차)
            param_grid = { 'C': stats.loguniform(1e-1, 1e0),"max_iter":stats.randint(100,500) }
        elif model_name == 'KNeighborsClassifier':#'algorithm':'auto' # p:1 manhattan_distance, 2 euclidean_distance 
            param_grid ={'n_neighbors':[5,10],'p':[1,2]}
        elif model_name == 'SGDClassifier':#early_stopping:False,'learning_rate': 'optimal','penalty': 'l2','loss': 'hinge','eta0':[0.0,1.0],
            param_grid = { 'alpha':stats.loguniform(1e-4,1e0),'l1_ratio':stats.uniform(0,1),'average':[True,False],
                            'epsilon':stats.loguniform(1e-1, 1e1),'max_iter':stats.randint(100,500)}
        elif model_name == 'DecisionTreeClassifier': 
            if(data_name=='project'):param_grid = {"max_depth": stats.randint(5, 50), "max_features": stats.randint(2, 24),
                "min_samples_split": stats.randint(10, 100), "min_samples_leaf": stats.randint(10, 100),"criterion": ["gini", "entropy"]}
        elif model_name == 'RandomForestClassifier': # n_estimators=100
            if(data_name=='project'):param_grid = {"max_depth":  stats.randint(5, 50),"max_features":stats.randint(2, 24),
            "min_samples_split":stats.randint(10,100),"min_samples_leaf":stats.randint(10,100),"bootstrap":[True,False],"criterion":["gini","entropy"]}
        elif model_name == 'GradientBoostingClassifier':#속도느림 #n_estimators=100 learning_rate:0.1 subsample:1.0 ccp_alpha:0.0 n_features
            if(data_name=='project'):param_grid={ 'max_depth' : stats.randint(5, 50),'min_samples_leaf': stats.randint(10,500),
                                                   'max_features':  stats.randint(5,24) }
        elif model_name == 'XGBClassifier': #min_child_samples, num_leaves 없음 #n_estimators=100 learning_rate:0.1 
            if(data_name=='project'):param_grid={'n_estimators': stats.randint(200, 1000),'learning_rate': stats.uniform(0.01, 0.6),
            'subsample':stats.uniform(0.3, 0.9),"max_depth":stats.randint(5,50),'colsample_bytree':stats.uniform(0.5,0.9),'min_child_weight':stats.loguniform(1e-3, 1e3) }
        elif model_name == 'LGBMClassifier':#n_estimators=100 learning_rate:0.1
            if(data_name=='project'):param_grid={'n_estimators': stats.randint(200, 1000),'learning_rate': stats.uniform(0.01, 0.6),
            'min_child_samples':stats.randint(10,500),'colsample_bytree':stats.uniform(loc=0.4,scale=0.6),'num_leaves': stats.randint(32,1000),
                                    'subsample': stats.uniform(loc=0.2, scale=0.8), 'min_child_weight': stats.loguniform(1e-3, 1e3),
                                      'reg_alpha': stats.loguniform(1e-3, 1e2),'reg_lambda': stats.loguniform(1e-3, 1e3)}   

        # reg
        elif model_name == 'LinearRegression':
            param_grid  = {'copy_X': [True, False], 'fit_intercept': [True,False] }
        elif model_name == 'LinearSVR': 
            param_grid = {'C':stats.loguniform(1e-1,1e0),'intercept_scaling':loguniform(1e-1,1e0),"max_iter":stats.randint(100,500)}
        elif model_name == 'SGDRegressor':
            param_grid = {'alpha':stats.loguniform(1e-4,1e0),'l1_ratio':stats.uniform(0,1),'average':[True,False],
                            'epsilon':stats.loguniform(1e-1, 1e1),'max_iter':stats.randint(100,500)}
        elif model_name == 'Ridge': 
            param_grid = {'alpha': stats.loguniform(1e-4,1e0),'fit_intercept': [True,False] } 
        elif model_name == 'Lasso': 
            param_grid = {'alpha': stats.loguniform(1e-4,1e0),'fit_intercept': [True,False] } 
        elif model_name == 'ElasticNet': # if L1 a, if L2 b  => alpha = a+b, l1_ratio = a/(a+b)  # if l1_ratio=1 => a=1, b=0
            param_grid = {'alpha': stats.loguniform(1e-4,1e0),'l1_ratio': stats.uniform(0,1)} 
        elif model_name == 'DecisionTreeRegressor':
            param_grid = {"max_depth": stats.randint(5, 50), "max_features": stats.randint(2, 24),
              "min_samples_split": stats.randint(10, 100), "min_samples_leaf": stats.randint(10, 100)}
        elif model_name == 'RandomForestRegressor': # cpu 병렬지원(n_jobs=-1, 100% 가동)
            param_grid = {"max_depth":  stats.randint(5, 50),"max_features":stats.randint(2, 24),"min_samples_split":stats.randint(10,100),
                          "min_samples_leaf":stats.randint(10,100)}
        elif model_name == 'GradientBoostingRegressor': # param많으면 학습시간이 많이 걸림(순차 시행, cpu 1개만 수행)
            param_grid={ 'max_depth' : stats.randint(5, 50),'min_samples_leaf': stats.randint(10,500),'max_features':  stats.randint(5,24)}
        elif model_name == 'XGBRegressor': #min_child_samples없음 # num_leaves 없음 # cpu 병렬지원(n_jobs=-1, auto) # 시간소요
            param_grid={'n_estimators': stats.randint(200,1000),'learning_rate': stats.uniform(0.01,0.6),'subsample':stats.uniform(0.2,0.8),
                "max_depth":stats.randint(5,50),'colsample_bytree':stats.uniform(0.5,0.9),'min_child_weight':stats.loguniform(1e-3, 1e3) }
        elif model_name == 'LGBMRegressor':  # cpu 병렬지원(n_jobs=-1, 100% 가동) # 매우빠름
            param_grid={'n_estimators':stats.randint(200,1000),'learning_rate':stats.uniform(0.01,0.6),'min_child_samples':stats.randint(10,500),
                'colsample_bytree':stats.uniform(loc=0.4,scale=0.6),'num_leaves': stats.randint(32,1000),'subsample':stats.uniform(0.2,0.8),
                'min_child_weight':stats.loguniform(1e-3,1e3),'reg_alpha':stats.loguniform(1e-3,1e2),'reg_lambda':stats.loguniform(1e-3,1e3)}
        else:
            param_grid = None
        return param_grid
        
    elif (grid == 3) : #GridSearch     
        # clf
        if model_name == 'LogisticRegression': # "solver":['lbfgs','auto']
            param_grid  = {'C': [0.1,0.2,0.3],"l1_ratio": [0.4,0.6,0.8],"max_iter":[400,500,600]}
        elif model_name == 'LinearSVC': # 'loss' ['hinge',''] # C 커지면 곡선 -> 직선 # tol(허용오차)
            param_grid ={'C': [0.2,0.3,0.4],"max_iter":[200,300,400] }
        elif model_name == 'KNeighborsClassifier':#'algorithm':'auto' # p:1 manhattan_distance, 2 euclidean_distance 
            param_grid ={'n_neighbors':[5,10],'p':[1,2]}
        elif model_name == 'SGDClassifier':#early_stopping:False,'learning_rate': 'optimal','penalty': 'l2','loss': 'hinge','eta0':[0.0,1.0],
            param_grid ={'alpha': [0.0001,0.01,0.1],"l1_ratio":[0.1,0.2,0.3],'epsilon':[0.1,1,10],'max_iter':[100,300,500]} # 빠름
        elif model_name == 'DecisionTreeClassifier': 
            if(data_name=='pima'):param_grid={'max_depth':[5,10,15],'min_samples_leaf':[10,100,500],'min_samples_split':[10,15,20],'max_features':[3,5,8]}
            if(data_name=='customer'):param_grid={'max_depth':[12,16,20],'min_samples_leaf':[10,16,20],'max_features':[100,200,300]}
            if(data_name=='credit'):param_grid={'max_depth':[12,18,24],'min_samples_leaf':[10,50,100],'max_features':[100,500,1000]}
            if(data_name=='project'):param_grid={'max_depth':[15,20,25],'min_samples_leaf':[50,100,150],'max_features':[4,8,12]}
        elif model_name == 'RandomForestClassifier': # n_estimators=100
            if(data_name=='pima'):param_grid={'max_depth':[5,10,15],'min_samples_leaf':[10,100,500],'min_samples_split':[10,15,20],'max_features':[3,5,8]}
            if(data_name=='customer'):param_grid={'max_depth':[12,16,20],'min_samples_leaf':[10,100,500],'max_features':[100,200,300]}
            if(data_name=='credit'):param_grid={'max_depth':[12,18,24],'min_samples_leaf':[10,50,100],'max_features':[100,500,1000]}
            if(data_name=='project'):param_grid={'max_depth':[15,20,25],'min_samples_leaf':[20,40,60],'min_samples_split':[20,40,60]}
        elif model_name == 'GradientBoostingClassifier':#속도느림 #n_estimators=100 learning_rate:0.1 subsample:1.0 ccp_alpha:0.0 n_features
            if(data_name=='pima'):param_grid={'max_depth':[10],'min_samples_leaf':[10,15],'min_samples_split':[10,20],'max_features':[3,5,8]}
            if(data_name=='customer'):param_grid={'max_depth':[16],'min_samples_leaf':[10,100],'max_features':[100,200,300]}
            if(data_name=='credit'):param_grid={'max_depth':[18],'min_samples_leaf':[10,100],'max_features':[10,20]}
            if(data_name=='project'):param_grid={'max_depth':[5,10,20],'min_samples_leaf':[100,200],'max_features':[5,10,15]}
        elif model_name == 'XGBClassifier': #min_child_samples, num_leaves 없음 #n_estimators=100 learning_rate:0.1 
            if(data_name=='pima'):param_grid={'max_depth':[5,10,15],'min_child_weight':[1,3],'colsample_bytree':[0.5,0.75,1],
                                  'subsample':[0.8,1],'reg_alpha': [0,0.5],'reg_lambda':[0.5,1]}
            if(data_name=='customer'):param_grid={'max_depth':[12,16,20],'min_child_weight':[1,3],'colsample_bytree':[0.5,0.75,1],
                                  'subsample':[0.8,1],'reg_alpha': [0,0.5],'reg_lambda':[0.5,1]}
            if(data_name=='credit'):param_grid={'max_depth':[18,36],'min_child_weight':[1,3],'colsample_bytree':[0.5,1]}
                                  #'subsample':[0.8,1]} #'reg_alpha': [0,0.5],'reg_lambda':[0.5,1]}
            if(data_name=='project'):param_grid={'n_estimators':[400,500,600],'learning_rate':[0.03,0.05,0.07],'max_depth':[6,15,30],
             'min_child_weight':[0.1,1,3],'colsample_bytree':[0.5,0.75,0.95],'subsample':[0.5,0.7,0.9],'reg_alpha':[0,0.5,1],'reg_lambda':[0,0.5,1]}
        elif model_name == 'LGBMClassifier':#n_estimators=100 learning_rate:0.1
            if(data_name=='pima'):param_grid={'max_depth':[128,160],'min_child_samples':[10,100,500],'colsample_bytree': [0.5,0.75,1],
                                  'num_leaves':[32,128,512],'subsample':[0.8,1],'reg_alpha': [0,0.5,1],'reg_lambda':[0,0.5,1]}
            if(data_name=='customer'):param_grid={'max_depth':[128,512],'min_child_samples':[10,100,500],'colsample_bytree': [0.5,0.75,1],
                                  'num_leaves':[32,128,512],'subsample':[0.8,1],'reg_alpha': [0,0.5,1],'reg_lambda':[0,0.5,1]}
            if(data_name=='credit'):param_grid={'max_depth':[128,512,1024],'min_child_samples':[10,100,1000],'colsample_bytree': [0.5,0.75,1],
                                  'num_leaves':[32,512,2048],'subsample':[0.8,1],'reg_alpha': [0,0.5,1],'reg_lambda':[0,0.5,1]}
            if(data_name=='project'):param_grid={'n_estimators':[400,500,600],'learning_rate':[0.03,0.05,0.07], 'min_child_samples':[100,150,200],
              'colsample_bytree':[0.5,0.75,1],'num_leaves':[50,100,150],'subsample':[0.6,0.8,1],'reg_alpha': [0,0.5,1],'reg_lambda':[0,0.5,1]}
                
        # reg
        elif model_name == 'LinearRegression':
            param_grid  = {'n_jobs' : [-1, 20 ], 'fit_intercept' : [True, False]}
        elif model_name == 'LinearSVR': # degree, C, kernal, epsilon, gamma , tol(허용오차) # degree 차원  # C 커지면 곡선 -> 직선 # gamma 커지면 과적합
            param_grid ={'C': [1, 10], 'max_iter' : [100, 1000]}
        elif model_name == 'SGDRegressor':#early_stopping:False,'learning_rate': 'optimal','penalty': 'l2','loss': 'hinge','eta0':[0.0,1.0],
            param_grid ={'alpha': [0.0001,0.1],"l1_ratio":[0.01,0.5],'epsilon':[0.1,1],'max_iter':[100,500]}
        elif model_name == 'Ridge': # L2규제(feature 가중치(베타)최소화)
            param_grid ={'alpha': [0.01, 0.1, 1, 10, 100, 500], 'solver':['auto','svd']}
        elif model_name == 'Lasso': # L1규제 (feature 가중치 제거)
            param_grid ={'alpha': [0.01, 0.1, 1, 10, 100, 500], 'max_iter' : [100, 1000]}
        elif model_name == 'ElasticNet': # if L1 a, if L2 b  => alpha = a+b, l1_ratio = a/(a+b)  # if l1_ratio=1 => a=1, b=0
            param_grid ={'alpha': [0.1, 1, 10, 100, 500], "l1_ratio": [0.1, 0.3, 0.5, 0.7, 0.9]}
        elif model_name == 'DecisionTreeRegressor':
            if (data_name=='bostonHouse'):param_grid={'max_depth':[5,8,10],'min_samples_leaf':[4,8,12,18],'min_samples_split':[2,8,16,50]}
            if (data_name=='bike'):param_grid={'max_depth':[8,13,20],'min_samples_leaf':[5,10,30,50,100 ],'min_samples_split':[5,10,30,50,100]}
            if (data_name=='AmesHouse'):param_grid={'max_depth':[5,10,20],'min_samples_leaf':[5,10,20,50],'min_samples_split':[5,10,20,50]}            
            if (data_name=='project'):param_grid={'max_depth':[15,22,30],'min_samples_leaf':[100,200,300],'min_samples_split':[200,500,1000]}
        elif model_name == 'RandomForestRegressor': # cpu 병렬지원(n_jobs=-1, 100% 가동)
            if (data_name=='bostonHouse'):param_grid={'max_depth':[5,8,10],'min_samples_leaf':[4,8,12,18 ],'min_samples_split':[2,8,16,20,50]}
            if (data_name=='bike'):param_grid={'max_depth':[8,13,20],'min_samples_leaf':[5,10,30,50,100],'min_samples_split':[5,10,30,50,100]}
            if (data_name=='AmesHouse'):param_grid={'max_depth':[5,10,20],'min_samples_leaf':[5,10,20,50],'min_samples_split':[5,10,20,50]}
        elif model_name == 'GradientBoostingRegressor': # param많으면 학습시간이 많이 걸림(순차 시행, cpu 1개만 수행)
            if (data_name=='bostonHouse'):param_grid= {'max_depth':[8],'min_samples_leaf':[8,36],'min_samples_split':[8,36]}
            if (data_name=='bike'):param_grid={'max_depth':[13],'min_samples_leaf':[10,50],'min_samples_split':[10,50]}
            if (data_name=='AmesHouse'):param_grid= {'max_depth':[10], 'min_samples_leaf':[10,20],'min_samples_split':[10,20]}
        elif model_name == 'XGBRegressor': #min_child_samples없음 # num_leaves 없음 # cpu 병렬지원(n_jobs=-1, auto) # 시간소요
            if (data_name=='bostonHouse'):param_grid={'max_depth':[5,8,10],'min_child_weight':[0.1,1,5],'subsample':[0.6,1]}
            if (data_name=='bike'):param_grid={'max_depth':[5,13,20],'min_child_weight':[0.1,1,5],'subsample':[0.6,1]}
            if (data_name=='AmesHouse'):param_grid={'max_depth':[5,10,20],'min_child_weight':[0.1,1,5],'subsample':[0.6,0.8,1]}
        elif model_name == 'LGBMRegressor':  # cpu 병렬지원(n_jobs=-1, 100% 가동) # 매우빠름
            if (data_name=='bostonHouse'):param_grid={'max_depth':[128,160],'min_child_samples':[20,60,100],'num_leaves':[32,64,128],'subsample':[0.6,0.8,1]}
            if (data_name=='bike'):param_grid={'max_depth':[64,128,256,512],'min_child_samples':[20,60,100],'num_leaves':[32,64,128],'subsample':[0.6,0.8,1]}
            if (data_name=='AmesHouse'):param_grid={'max_depth':[64,128,256],'min_child_samples':[20,60,100],'num_leaves':[32,64,128],'subsample':[0.6,0.8,1]}
                
        else:
            param_grid = None
        return param_grid

In [ ]:
# 선형회귀모델을 위한 scaler # 표준정규분포(Standard), 최대값/최소값(MinMax), 로그변환(Log)
# p_degree는 다항식 특성을 추가할 때 적용. p_degree는 2이상 부여하지 않음. 
def get_scaled_data(input_data=None, scaler=None, p_degree=None):
    if scaler == 'Standard':
        scaled_data = StandardScaler().fit_transform(input_data)
    elif scaler == 'MinMax':
        scaled_data = MinMaxScaler().fit_transform(input_data)
    elif scaler == 'Log':
        scaled_data = np.log1p(input_data)
    else:
        scaled_data = input_data
    if p_degree != None:
        scaled_data = PolynomialFeatures(degree=p_degree, include_bias=False).fit_transform(scaled_data)
    return scaled_data